# MM Binding with cavity correction to csv files

In [35]:
# Loads stuff
import sys
sys.path.insert(0,"/home/lg3u19/OnePy")
import onetep_v0_1 as op
import pathlib
import pandas as pd
pd.set_option('display.precision',3) 
import copy

In [36]:
csv_target_path = './ProcessedData/Binding'

In [37]:
snapshots_5 = ['24801', '32401', '17201', '9601','2001'] 
snapshots_10 = ['24801', '32401', '17201', '13401', '21001', '28601', '9601', '5801','36201','2001'] 
snapshots_25 = [2001, 3521, 5041, 6561, 8081, 9601, 11121, 12641, 14161, 15681, 17201, 18721, 20241, 21761, 23281, 24801, 26321, 27841, 29361, 30881, 32401, 33921, 35441, 36961, 38481]
snapshots_25 = [str(x) for x in snapshots_25]
snapshots_50 = [2001, 2761, 3521, 4281, 5041, 5801, 6561, 7321, 8081, 8841, 9601, 10361, 11121, 11881, 12641, 13401, 14161, 14921, 15681, 16441, 17201, 17961, 18721, 19481, 20241, 21001, 21761, 22521, 23281, 24041, 24801, 25561, 26321, 27081, 27841, 28601, 29361, 30121, 30881, 31641, 32401, 33161, 33921, 34681, 35441, 36201, 36961, 37721, 38481, 39241]
snapshots_50 = [str(x) for x in snapshots_50]

## Function for MM Binding energies

In [38]:
# Creat a dictionary in which each ligand has a DF of all data from .csv files for every snapshot
mm_dict = {}
for ligand in ['phenol','methylphenol','catechol','fluoroaniline','hydroxyaniline']:
    # load data from file
    mm_data = pd.read_csv('MM-PBSA/'+ligand+'/MM-energies-solvation.csv',delimiter=';')
    # set first column as index and relable to snaphot
    mm_data = mm_data.set_index(mm_data.columns[0])
    mm_data.index.name = 'snapshot'
    # select only snaphsots and get ridd of rest of file
    mm_data = mm_data.loc['2001':'39963',:]
    # drop empyt columns 
    mm_data = mm_data.dropna(axis=1)
    # deal with odd formatting of methylphenol file
    if 'Unnamed: 13' in mm_data.columns:
        mm_data = mm_data.drop('Unnamed: 13',axis=1)
    # rename columns
    mm_data.columns=['comp-gas','comp-polar',
                        'comp-non-polar','comp-total','host-gas',
                        'host-polar','host-non-polar','host-total',
                        'lig-gas','lig-polar','lig-non-polar',
                        'lig-total','net-gas','net-polar',
                        'net-non-polar','net-total']
    # change data type to numeric
    mm_data = mm_data.apply(pd.to_numeric)
    # add df to dictionary
    mm_dict[ligand]=mm_data
# make sure i dont accidenatlly use temp variable mm_data
mm_data = False
    

In [43]:
def mm_binding_energy_clean_table(mm_data,snapshots):
    """ given df with mm data on all snapshots, produced df with energy components
    in standard format of qm calcs for easy comparison"""
    # select subset 
    if snapshots=='All':
        subset_df = mm_data.loc[:,:]
    else:
        subset_df = mm_data.loc[snapshots,:]
    # determine mm cav correction term
    subset_df['E']=subset_df['net-gas']
    subset_df['G_solv']=subset_df['net-polar']+subset_df['net-non-polar']
    subset_df['correction']=2*(subset_df['host-non-polar']-subset_df['comp-non-polar'])
    subset_df['G_solv_corrected']=subset_df['G_solv']+subset_df['correction']
    subset_df['G_bind']=subset_df['E']+subset_df['G_solv_corrected']
    return subset_df.loc[:,['E','G_solv','G_solv_corrected','G_bind']]

# Create csv with snapshot binding energy components for each ligand

In [44]:
csv_targe = './ProcessedData/BindingComponentsMM/'
for ligand in mm_dict.keys():
    temp_df = mm_binding_energy_clean_table(mm_dict[ligand],snapshots_25)
    filename = ligand+'_MM_25_corrected.csv'
    temp_df.to_csv(csv_targe+filename)